# Importing Library

In [47]:
import pandas as pd
import numpy as np

# Extract

# Extracting File from the Ipl Database

In [48]:
balls=pd.read_csv("data/raw/IPL_2008_2022.csv")

# Transform

## Batting

### Counting the Runs

In [49]:
Runs=balls.groupby(["ID","batter"])["batsman_run"].sum().reset_index().rename(columns={"batsman_run":"Batsman_Runs"})

### Couting the Balls

In [50]:
balls_count=balls[~balls["extra_type"].isin(["wides"])]
Balls=balls_count.groupby(["ID","batter"])["batsman_run"].count().reset_index().rename(columns={"batsman_run":"Balls"})

### Fours and Sixes

In [51]:
Fours=balls[balls["batsman_run"]==4].groupby(["ID","batter"])["batsman_run"].count().reset_index().rename(columns={"batsman_run":"Fours"})

In [52]:
Sixes=balls[balls["batsman_run"]==6].groupby(["ID","batter"])["batsman_run"].count().reset_index().rename(columns={"batsman_run":"Sixes"})

### Batting Perfomance

In [53]:
Batting=Runs.merge(Balls,on=["ID","batter"],how="left").merge(Fours,on=["ID","batter"],how="left").merge(Sixes,on=["ID","batter"],how="left")
Batting.fillna(0,inplace=True)

### Strike Rate

In [54]:
Batting["Strike_Rate"]=Batting["Batsman_Runs"]/Batting["Balls"]*100

In [55]:
Batting

,ID,batter,Batsman_Runs,Balls,Fours,Sixes,Strike_Rate
0,335982,AA Noffke,9,10.0,1.0,0.0,90.000000
1,335982,B Akhil,0,2.0,0.0,0.0,0.000000
2,335982,BB McCullum,158,73.0,10.0,13.0,216.438356
3,335982,CL White,6,10.0,0.0,0.0,60.000000
4,335982,DJ Hussey,12,12.0,1.0,0.0,100.000000
...,...,...,...,...,...,...,...
14224,1312200,SV Samson,14,11.0,2.0,0.0,127.272727
14225,1312200,Shubman Gill,45,43.0,3.0,1.0,104.651163
14226,1312200,TA Boult,11,7.0,0.0,1.0,157.142857
14227,1312200,WP Saha,5,7.0,1.0,0.0,71.428571


### Batting Dream 11 Score

In [56]:
def Batting_Score(row):
    score=row["Batsman_Runs"]
    score=score+(4*row["Fours"])+(6*row["Sixes"])
    if row["Batsman_Runs"]>25 and row["Batsman_Runs"]< 50:
        score=score+4
    elif  row["Batsman_Runs"]>=50 and row["Batsman_Runs"]< 75:
        score=score+8+4
    elif  row["Batsman_Runs"]>=75 and row["Batsman_Runs"]< 100:
        score=score+12+8+4
    elif row["Batsman_Runs"]>=100:
        score=score+16+12+8+4
    elif row["Batsman_Runs"]==0:
        score=score-2
    elif row["Balls"]>=10:
        if row["Strike_Rate"]>=170:
            score=score+6
        elif row["Strike_Rate"]>150.01 and row["Strike_Rate"]<170:
            score=score+4
        elif row["Strike_Rate"]>130 and row["Strike_Rate"]<150:
            score=score+2
        elif row["Strike_Rate"]>60 and row["Strike_Rate"]<70:
            score=score-2
        elif row["Strike_Rate"]>50 and row["Strike_Rate"]<59.99:
            score=score-4
        elif row["Strike_Rate"]<50:
            score=score-6
    return score
        
    

In [57]:
Batting["Bat_Score"]=Batting.apply(Batting_Score,axis=1)

In [58]:
Batting

,ID,batter,Batsman_Runs,Balls,Fours,Sixes,Strike_Rate,Bat_Score
0,335982,AA Noffke,9,10.0,1.0,0.0,90.000000,13.0
1,335982,B Akhil,0,2.0,0.0,0.0,0.000000,-2.0
2,335982,BB McCullum,158,73.0,10.0,13.0,216.438356,316.0
3,335982,CL White,6,10.0,0.0,0.0,60.000000,6.0
4,335982,DJ Hussey,12,12.0,1.0,0.0,100.000000,16.0
...,...,...,...,...,...,...,...,...
14224,1312200,SV Samson,14,11.0,2.0,0.0,127.272727,22.0
14225,1312200,Shubman Gill,45,43.0,3.0,1.0,104.651163,67.0
14226,1312200,TA Boult,11,7.0,0.0,1.0,157.142857,17.0
14227,1312200,WP Saha,5,7.0,1.0,0.0,71.428571,9.0


### Keeping Only ID,batter,Bat_Score

In [59]:
Batting=Batting[["ID","batter","Bat_Score"]]
Batting

,ID,batter,Bat_Score
0,335982,AA Noffke,13.0
1,335982,B Akhil,-2.0
2,335982,BB McCullum,316.0
3,335982,CL White,6.0
4,335982,DJ Hussey,16.0
...,...,...,...
14224,1312200,SV Samson,22.0
14225,1312200,Shubman Gill,67.0
14226,1312200,TA Boult,17.0
14227,1312200,WP Saha,9.0


## Bowling

### Runs against Bowler

In [60]:
Runs_conceded=balls.groupby(["ID","bowler"])["total_run"].sum().reset_index(name="runs_conceded")

### Dots by the Bowler

In [61]:
dot_balls = (
    balls[balls["total_run"] == 0]
    .groupby(["ID", "bowler"])
    .size()
    .reset_index(name="dot_balls")
)

### Wickets by Bowler

In [62]:
wickets = (
    balls[(balls["isWicketDelivery"] == 1) & (balls["kind"] != "run out")]
    .groupby(["ID", "bowler"])
    .size()
    .reset_index(name="wickets")
)

### Maiden Over by Bowler

In [63]:
over_runs = (
    balls.groupby(["ID", "bowler", "overs"])["total_run"]
      .sum()
      .reset_index(name="over_runs")
)
maidens = (
    over_runs[over_runs["over_runs"] == 0]
    .groupby(["ID", "bowler"])
    .size()
    .reset_index(name="maidens")
)

### Legal Balls Bowled by Bowler

In [64]:
balls_bowled=balls[~balls["extra_type"].isin(["wides", "noballs","penalty"])].groupby(["ID","bowler"]).size().reset_index(name="balls_bowled")


### Bowled/lbw Wickets

In [65]:
Bowled_lbw = (
    balls[
        (balls["isWicketDelivery"] == 1) &
        (balls["kind"].isin(['bowled', "lbw"]))
    ]
    .groupby(["ID","bowler"])
    .size()
    .reset_index(name="wickets_bowled_lbw")
)

### Bowling Perfomance

In [66]:
Bowling=Runs_conceded.merge(balls_bowled,on=["ID","bowler"],how="left").merge(dot_balls,on=["ID","bowler"],how="left").merge(wickets,on=["ID","bowler"],how="left")
Bowling=Bowling.merge(maidens,on=["ID","bowler"],how="left")
Bowling=Bowling.merge(Bowled_lbw,on=["ID","bowler"],how="left")
Bowling.fillna(0,inplace=True)
Bowling.rename(columns={"wickets_bowled_lbw":"Bowled/lbw"},inplace=True)

### Bowling Economy

In [67]:
Bowling["Economy"]=(Bowling["runs_conceded"]/Bowling["balls_bowled"])*6

### Bowling Score

In [68]:
def Bowling_score(row):
    score=0
    score=score+row["dot_balls"]+(30*row["wickets"]) + (8*row["Bowled/lbw"]) +(12*row["maidens"])
    if row["wickets"]>=3:
        score=score+4
    elif row["wickets"]>=4:
        score=score+8
    elif row["wickets"]>=5:
        score=score+12
    elif row["balls_bowled"]>=12:
        if row["Economy"]<5:
            score=score+6
        elif row["Economy"]>=5 and row["Economy"]<5.99:
            score=score+4
        elif row["Economy"]>6 and row["Economy"]<7:
            score=score+2
        elif row["Economy"]>10 and row["Economy"]<11:
            score=score-2
        elif row["Economy"]>11.01 and row["Economy"]<12:
            score=score-4
        elif row["Economy"]>12:
            score=score-6
    return score 

In [69]:
Bowling["Bowling_Score"]=Bowling.apply(Bowling_score,axis=1)

In [70]:
Bowling

,ID,bowler,runs_conceded,balls_bowled,dot_balls,wickets,maidens,Bowled/lbw,Economy,Bowling_Score
0,335982,AA Noffke,41,24.0,6.0,1.0,0.0,0.0,10.250000,34.0
1,335982,AB Agarkar,25,24.0,15.0,3.0,0.0,0.0,6.250000,109.0
2,335982,AB Dinda,9,18.0,11.0,2.0,0.0,1.0,3.000000,85.0
3,335982,CL White,24,6.0,0.0,0.0,0.0,0.0,24.000000,0.0
4,335982,I Sharma,13,18.0,10.0,1.0,0.0,1.0,4.333333,54.0
...,...,...,...,...,...,...,...,...,...,...
11218,1312200,R Sai Kishore,20,12.0,3.0,2.0,0.0,0.0,10.000000,63.0
11219,1312200,Rashid Khan,18,24.0,9.0,1.0,0.0,0.0,4.500000,45.0
11220,1312200,TA Boult,15,24.0,16.0,1.0,1.0,0.0,3.750000,64.0
11221,1312200,YS Chahal,20,24.0,10.0,1.0,0.0,0.0,5.000000,44.0


In [71]:
Bowling=Bowling[["ID","bowler","Bowling_Score"]]
Bowling

,ID,bowler,Bowling_Score
0,335982,AA Noffke,34.0
1,335982,AB Agarkar,109.0
2,335982,AB Dinda,85.0
3,335982,CL White,0.0
4,335982,I Sharma,54.0
...,...,...,...
11218,1312200,R Sai Kishore,63.0
11219,1312200,Rashid Khan,45.0
11220,1312200,TA Boult,64.0
11221,1312200,YS Chahal,44.0


## Fielding

### Filtering only the Caught,Run_Out,Stumped

In [72]:
Catch_Run_out=balls[balls["kind"].isin(['caught','run out','stumped'])]

### Catches

In [73]:
Catch=Catch_Run_out[Catch_Run_out["kind"]=="caught"]

In [74]:
Catch=Catch_Run_out[Catch_Run_out["kind"]=="caught"].groupby(["ID","fielders_involved"])["isWicketDelivery"].sum().reset_index(name="Catches")

### Stumping

In [75]:
Stumping=Catch_Run_out[Catch_Run_out["kind"]=="stumped"].groupby(["ID","fielders_involved"])["isWicketDelivery"].sum().reset_index(name="Stumping")

### Run Out

In [76]:
Run_out=Catch_Run_out[Catch_Run_out["kind"]=="run out"].groupby(["ID","fielders_involved"])["isWicketDelivery"].sum().reset_index(name="Run_out")

### Fielding Perfomance

In [77]:
Fielding=Catch.merge(Stumping, on =["ID","fielders_involved"],how="outer").merge(Run_out,on =["ID","fielders_involved"],how="outer")
Fielding.fillna(0,inplace=True)

In [78]:
Fielding

,ID,fielders_involved,Catches,Stumping,Run_out
0,335982,BB McCullum,1.0,0.0,0.0
1,335982,CL White,1.0,0.0,0.0
2,335982,JH Kallis,1.0,0.0,0.0
3,335982,M Kartik,2.0,0.0,0.0
4,335982,P Kumar,1.0,0.0,0.0
...,...,...,...,...,...
6329,1304109,KL Rahul,0.0,0.0,1.0
6330,1304111,B Kumar,0.0,0.0,1.0
6331,1304111,T Natarajan,0.0,0.0,1.0
6332,1304113,F du Plessis,0.0,0.0,1.0


### Fielding Score

In [79]:
def Fielding_Score(row):
    score=0
    score=(8*row["Catches"])+(12*row["Stumping"])+(12*row["Run_out"])
    if row["Catches"]>=3:
        score=score+4
    return score

In [80]:
Fielding["Fielding_Score"]=Fielding.apply(Fielding_Score,axis=1)

In [81]:
Fielding

,ID,fielders_involved,Catches,Stumping,Run_out,Fielding_Score
0,335982,BB McCullum,1.0,0.0,0.0,8.0
1,335982,CL White,1.0,0.0,0.0,8.0
2,335982,JH Kallis,1.0,0.0,0.0,8.0
3,335982,M Kartik,2.0,0.0,0.0,16.0
4,335982,P Kumar,1.0,0.0,0.0,8.0
...,...,...,...,...,...,...
6329,1304109,KL Rahul,0.0,0.0,1.0,12.0
6330,1304111,B Kumar,0.0,0.0,1.0,12.0
6331,1304111,T Natarajan,0.0,0.0,1.0,12.0
6332,1304113,F du Plessis,0.0,0.0,1.0,12.0


In [82]:
Fielding=Fielding[["ID","fielders_involved","Fielding_Score"]]

## Merging Batting and Bowling Table

In [83]:
Batting_Bowling=pd.merge(Batting,Bowling,left_on=["ID","batter"],right_on=["ID","bowler"],how="outer")
Batting_Bowling

,ID,batter,Bat_Score,bowler,Bowling_Score
0,335982,AA Noffke,13.0,AA Noffke,34.0
1,335982,B Akhil,-2.0,NaN,NaN
2,335982,BB McCullum,316.0,NaN,NaN
3,335982,CL White,6.0,CL White,0.0
4,335982,DJ Hussey,16.0,NaN,NaN
...,...,...,...,...,...
20018,1312200,NaN,NaN,Mohammed Shami,49.0
20019,1312200,NaN,NaN,R Sai Kishore,63.0
20020,1312200,NaN,NaN,Rashid Khan,45.0
20021,1312200,NaN,NaN,YS Chahal,44.0


## Merging Batting,Bowling and Fielding

In [84]:
Batting_Bowling_Fielding=pd.merge(Batting_Bowling,Fielding,left_on=["ID","batter"],right_on=["ID","fielders_involved"],how="outer")

In [85]:
Batting_Bowling_Fielding["Player"]=Batting_Bowling_Fielding["batter"].combine_first(Batting_Bowling_Fielding["bowler"])


In [86]:
Batting_Bowling_Fielding["Players"]=Batting_Bowling_Fielding["Player"].combine_first(Batting_Bowling_Fielding["fielders_involved"])
Batting_Bowling_Fielding

,ID,batter,Bat_Score,bowler,Bowling_Score,fielders_involved,Fielding_Score,Player,Players
0,335982,AA Noffke,13.0,AA Noffke,34.0,NaN,NaN,AA Noffke,AA Noffke
1,335982,B Akhil,-2.0,NaN,NaN,NaN,NaN,B Akhil,B Akhil
2,335982,BB McCullum,316.0,NaN,NaN,BB McCullum,8.0,BB McCullum,BB McCullum
3,335982,CL White,6.0,CL White,0.0,CL White,8.0,CL White,CL White
4,335982,DJ Hussey,16.0,NaN,NaN,NaN,NaN,DJ Hussey,DJ Hussey
...,...,...,...,...,...,...,...,...,...
21678,1304073,NaN,NaN,NaN,NaN,PWH de Silva,12.0,NaN,PWH de Silva
21679,1304086,NaN,NaN,NaN,NaN,AS Joseph,12.0,NaN,AS Joseph
21680,1304094,NaN,NaN,NaN,NaN,R Dhawan,12.0,NaN,R Dhawan
21681,1304111,NaN,NaN,NaN,NaN,B Kumar,12.0,NaN,B Kumar


In [87]:
Players=Batting_Bowling_Fielding.drop(columns=["batter","bowler","fielders_involved","Player"])

In [88]:
Players=Players[["ID","Players","Bat_Score","Bowling_Score","Fielding_Score"]].fillna(0)

### Finding the Total Score

In [89]:
Players["Score"]=Players["Bat_Score"]+Players["Bowling_Score"]+Players["Fielding_Score"]

In [90]:
Players

,ID,Players,Bat_Score,Bowling_Score,Fielding_Score,Score
0,335982,AA Noffke,13.0,34.0,0.0,47.0
1,335982,B Akhil,-2.0,0.0,0.0,-2.0
2,335982,BB McCullum,316.0,0.0,8.0,324.0
3,335982,CL White,6.0,0.0,8.0,14.0
4,335982,DJ Hussey,16.0,0.0,0.0,16.0
...,...,...,...,...,...,...
21678,1304073,PWH de Silva,0.0,0.0,12.0,12.0
21679,1304086,AS Joseph,0.0,0.0,12.0,12.0
21680,1304094,R Dhawan,0.0,0.0,12.0,12.0
21681,1304111,B Kumar,0.0,0.0,12.0,12.0


In [ ]:
players.to_csv("outputs/Dream_11_points_table.csv")